In [ ]:
# Import Libraries
import pandas as pd 
import numpy as np 
import yfinance as yf
from datetime import datetime, timedelta
import os
import talib

### Stock Data Pull

In [ ]:
# Get S&P500 symbols ranked by market cap 
sp500_by_marketcap = pd.read_csv('SP500_By_MarketCap.csv')
sp500_by_marketcap.head()

,Symbol,Name,Industry,Market Cap
0,AAPL,Apple Inc.,Electronic Technology,"$ 2,986,128,347,290.24"
1,MSFT,Microsoft Corporation,Technology Services,"$ 2,513,296,516,647.36"
2,GOOG,Alphabet Inc.,Technology Services,"$ 1,927,101,773,229.48"
3,GOOGL,Alphabet Inc.,Technology Services,"$ 1,923,705,624,039.54"
4,AMZN,"Amazon.com, Inc.",Retail Trade,"$ 1,728,404,755,739.39"


In [ ]:
# Get tickers in a list

tickers = sp500_by_marketcap['Symbol'].head(50).tolist()
print(tickers)

['AAPL', 'MSFT', 'GOOG', 'GOOGL', 'AMZN', 'TSLA', 'FB', 'NVDA', 'BRK-B', 'JPM', 'UNH', 'V', 'JNJ', 'HD', 'WMT', 'PG', 'BAC', 'MA', 'PFE', 'DIS', 'AVGO', 'XOM', 'ACN', 'CSCO', 'NFLX', 'NKE', 'LLY', 'KO', 'TMO', 'CRM', 'COST', 'ABT', 'ABBV', 'PEP', 'ORCL', 'CMCSA', 'CVX', 'PYPL', 'DHR', 'VZ', 'INTC', 'QCOM', 'WFC', 'MCD', 'MRK', 'UPS', 'T', 'AMD', 'NEE', 'MS']


In [ ]:
# Set timeframe 
delta = 365
end = datetime.now()
start = datetime.now() - timedelta(delta)

In [ ]:
# Get stock data from yfinance 
df_stock_data=pd.DataFrame()
for ticker in tickers:
    ticker_df = yf.download(ticker, start=start, end=end,interval="1D")
    ticker_df = ticker_df.reset_index()
    ticker_df["Stock"] = ticker
    # Creating a unique key by combining Ticker and Publish date
    ticker_df["unique_key"] = ticker_df["Stock"]+ticker_df["Date"].astype(str)
    df_stock_data =pd.concat([df_stock_data,ticker_df],axis=0)
df_stock_data = df_stock_data.set_index("unique_key")
df_stock_data.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,Date,Open,High,Low,Close,Adj Close,Volume,Stock
unique_key,,,,,,,,
AAPL2021-04-15,2021-04-15,133.820007,135.000000,133.639999,134.500000,133.705322,89347100,AAPL
AAPL2021-04-16,2021-04-16,134.300003,134.669998,133.279999,134.160004,133.367325,84922400,AAPL
AAPL2021-04-19,2021-04-19,133.509995,135.470001,133.339996,134.839996,134.043320,94264200,AAPL
AAPL2021-04-20,2021-04-20,135.020004,135.529999,131.809998,133.110001,132.323532,94812300,AAPL
AAPL2021-04-21,2021-04-21,132.360001,133.750000,131.300003,133.500000,132.711227,68847100,AAPL


In [ ]:
# Create "Returns" column 
returns_df = pd.DataFrame()
returns_df = pd.DataFrame(df_stock_data["Adj Close"].pct_change().shift(-1))
returns_df = returns_df.rename(columns = {"Adj Close": "Returns"})
returns_df

,Returns
unique_key,
AAPL2021-04-15,-0.002528
AAPL2021-04-16,0.005069
AAPL2021-04-19,-0.012830
AAPL2021-04-20,0.002930
AAPL2021-04-21,-0.011685
...,...
MS2022-04-08,-0.000476
MS2022-04-11,-0.001071
MS2022-04-12,0.002383


In [ ]:
df_stock_data = pd.concat([df_stock_data,returns_df],join = 'outer',axis = 1)
df_stock_data

,Date,Open,High,Low,Close,Adj Close,Volume,Stock,Returns
unique_key,,,,,,,,,
AAPL2021-04-15,2021-04-15,133.820007,135.000000,133.639999,134.500000,133.705322,89347100,AAPL,-0.002528
AAPL2021-04-16,2021-04-16,134.300003,134.669998,133.279999,134.160004,133.367325,84922400,AAPL,0.005069
AAPL2021-04-19,2021-04-19,133.509995,135.470001,133.339996,134.839996,134.043320,94264200,AAPL,-0.012830
AAPL2021-04-20,2021-04-20,135.020004,135.529999,131.809998,133.110001,132.323532,94812300,AAPL,0.002930
AAPL2021-04-21,2021-04-21,132.360001,133.750000,131.300003,133.500000,132.711227,68847100,AAPL,-0.011685
...,...,...,...,...,...,...,...,...,...
MS2022-04-08,2022-04-08,83.529999,84.769997,82.959999,84.059998,84.059998,6935400,MS,-0.000476
MS2022-04-11,2022-04-11,83.580002,85.820000,83.019997,84.019997,84.019997,8290400,MS,-0.001071
MS2022-04-12,2022-04-12,83.760002,85.980003,83.540001,83.930000,83.930000,8057300,MS,0.002383


### Techincal Indicator Preparation

In [ ]:
indicator_df = pd.DataFrame()

for ticker in tickers:
  df = df_stock_data[df_stock_data['Stock']== ticker]
  # Simple Moving Average for 90 day period
  sma = pd.DataFrame(talib.SMA(df['Adj Close'], 90)).rename(columns={0:'sma'})
  # Exponential Moving Average for 90 day period
  ema = pd.DataFrame(talib.EMA(df['Adj Close'], timeperiod=90)).rename(columns={0:'ema'})
  # Momentum for 90 day period
  mom = pd.DataFrame(talib.MOM(df['Adj Close'],timeperiod=90)).rename(columns={0:'mom'})
  # Average Directional Movement
  adx = pd.DataFrame(talib.ADX(df['High'],df['Low'],df['Adj Close'],timeperiod=90)).rename(columns={0:'adx'})
  # Normalized Average True Range for 90 day period
  natr = pd.DataFrame(talib.NATR(df['High'],df['Low'],df['Adj Close'],timeperiod=90)).rename(columns={0:'natr'})
  # Linear Regression for 90 day period
  linreg = pd.DataFrame(talib.LINEARREG(df['Adj Close'], timeperiod=90)).rename(columns={0:'linreg'})
  # Hilbert Transform Trend vs Cycle Mode
  httrend = pd.DataFrame(talib.HT_TRENDMODE(df['Adj Close'])).rename(columns={0:'httrend'})
  # Relative Strength for 90 day period
  rsi = pd.DataFrame(talib.RSI(df['Adj Close'], timeperiod=90)).rename(columns={0:'rsi'})
  # Typical Price 
  typprice = pd.DataFrame(talib.TYPPRICE(df['High'],df['Low'],df['Adj Close'])).rename(columns={0:'typprice'})
  # MFI- Money Flow Index
  mfi = pd.DataFrame(talib.MFI(df['High'],df['Low'], df['Adj Close'],df['Volume'],timeperiod=90)).rename(columns={0:'mfi'})
  # ADOSC-Chaikin A/D Oscillator
  adosc = pd.DataFrame(talib.ADOSC(df['High'],df['Low'],df['Adj Close'],df['Volume'],fastperiod=3,slowperiod=10)).rename(columns={0:'adosc'})
  # Hilbert Transform - Dominant Cycle Period
  domcycle= pd.DataFrame(talib.HT_DCPERIOD(df['Adj Close'])).rename(columns={0:'domcycle'})
  # Hilbert Transform - Dominant Cycle Phase
  dom_cyc_phs = pd.DataFrame(talib.HT_DCPHASE(df['Adj Close'])).rename(columns={0:'dom_cyc_phs'})
  #bringing all indicators in one DataFrame
  ticker_indicator_df=pd.concat([sma,
                                ema,
                                mom,
                                adx,
                                natr,
                                linreg,
                                httrend,
                                rsi,
                                typprice,
                                mfi,
                                adosc,
                                domcycle,
                                dom_cyc_phs],axis=1)
  indicator_df = pd.concat([indicator_df,ticker_indicator_df],axis=0)


In [ ]:
indicator_df.to_csv('indicator.csv')